# Calculating Revenue Growth Percentage % using SQL, Pandas and Python3 Functions

## Month-over-Month (MoM) Growth Percentage 

This is a key financial metric that shows the rate at which a value (such as revenue, profit, or customer count) has increased or decreased compared to the previous month.

## Why is it important:

__Short-Term Trend Tracking:__ Month Over Month revenue growth gives a timely snapshot of performance, helping finacial teams understand recent changes without waiting for quarterly or yearly trends.

__Early Detection of Issues or Wins:__ A sharp drop or spike in Month Over Month revenue, expenses, or customer churn can be an early indicator of problems or successes needing immediate attention.

__Credit/Loan Default Volume Metric:__ A Month Over Month revenue growth change in loan defaults or charge-offs. A sudden rise can signal portfolio risk escalation or underwriting weaknesses.

__Credit Utilization Ratios:__ Increasing utilization rates can indicate financial stress among borrowers and signal future repayment problems.

For example if a bank sees a 10% Month Over Month increase in digital loan applications. This might:

* Prompt deeper analysis into customer behavior.

* Signal growing digital engagement.

* Justify additional investment in the mobile platform.




## Method 1: SQL Server CTE to Calculate Revenue Growth Percentage %

```sql
-- month over month revenue growth
-- revenue growth % = (current month revenue - previous month revenue X 100) / previous month revenue

 --Month-over-month revenue growth
with month_rev as (
    select 
        month(order_date) as month,
        coalesce(sum(order_amount),0) as monthly_revenue
    from orders
    group by month(order_date)
)
select 
    month,
    monthly_revenue,
    coalesce(lag(monthly_revenue) over(order by month),0) as previous_month_revenue,
    coalesce(round(monthly_revenue - lag(monthly_revenue) over(order by month) * 100 / 
    nullif(lag(monthly_revenue) over(order by month),0), 2), 0) as revenue_growth_percent
from month_rev;

```


## Method 2: Pandas - Calculate Revenue Growth Percentage %

In [1]:
import pandas as pd

orders = pd.read_csv('/Users/johnryan/Downloads/Documents/Data Sets/orders.csv')
orders.head(5)

,order_id,customer_id,order_date,order_amount
0,1,1006,2023-12-15,405.81
1,2,1016,2023-08-01,953.18
2,3,1019,2023-08-13,745.39
3,4,1003,2023-01-31,618.73
4,5,1004,2023-05-07,198.22


In [2]:
# replicating at the SQL server CTE above to calculate month over month revenue growth
#get the month from order_date and group by month

orders['order_date'] = pd.to_datetime(orders['order_date'])
monthly_revenue = orders.groupby(orders['order_date'].dt.month)['order_amount'].sum().reset_index(name ='monthly_revenue')
monthly_revenue.rename(columns={'order_date': 'month'}, inplace=True)



# Now the CTE can be used to help with calcualtion of revenue growth
# Calculate previous month revenue .shift(1) in pandas is the Python equivalent of the SQL LAG() window function

monthly_revenue['previous_month'] = monthly_revenue['monthly_revenue'].shift(1).fillna(0)
monthly_revenue['revenue_growth_percent'] = ((monthly_revenue['monthly_revenue'].fillna(0) - monthly_revenue['previous_month'].fillna(0)) 
/ monthly_revenue['previous_month']).round(1) * 100
monthly_revenue.head()

,month,monthly_revenue,previous_month,revenue_growth_percent
0,1,2051.37,0.00,inf
1,2,3225.72,2051.37,60.0
2,3,2441.02,3225.72,-20.0
3,4,5266.72,2441.02,120.0
4,5,2474.76,5266.72,-50.0


## Method 3: Python3 using functions - Calculate Revenue Growth Percentage %

In [8]:
#first step get current monthly revenue
def get_monthly_revenue(orders):
    return orders.groupby(orders['order_date'].dt.month)['order_amount'].sum().reset_index(name='monthly_revenue')
monthly_revenue =  get_monthly_revenue(orders)
print(monthly_revenue)

    order_date  monthly_revenue
0            1          2051.37
1            2          3225.72
2            3          2441.02
3            4          5266.72
4            5          2474.76
5            6          5196.00
6            7          5013.02
7            8          6381.82
8            9          3361.34
9           10          5551.05
10          11          4853.43
11          12          3850.91


In [43]:
#Now use the above result set to calculate the revenue grwoth percentage
def get_monthly_revenue(orders):
    return orders.groupby(orders['order_date'].dt.month)['order_amount'].sum().reset_index(name='monthly_revenue')

monthly_revenue = get_monthly_revenue(orders)

def calculate_growth(monthly_revenue):
    monthly_revenue['prev_month_revenue'] = monthly_revenue['monthly_revenue'].shift(1).fillna(0) # like lag() function in sql
    monthly_revenue['revenue_growth_percent'] = ((monthly_revenue['monthly_revenue'].fillna(0) - monthly_revenue['prev_month_revenue'].fillna(0)) /
                                                 monthly_revenue['prev_month_revenue']).round(1) * 100
    return monthly_revenue
    
revenue_growth_percent = calculate_growth(monthly_revenue)
print(revenue_growth_percent)

    order_date  monthly_revenue  prev_month_revenue  revenue_growth_percent
0            1          2051.37                0.00                     inf
1            2          3225.72             2051.37                    60.0
2            3          2441.02             3225.72                   -20.0
3            4          5266.72             2441.02                   120.0
4            5          2474.76             5266.72                   -50.0
5            6          5196.00             2474.76                   110.0
6            7          5013.02             5196.00                    -0.0
7            8          6381.82             5013.02                    30.0
8            9          3361.34             6381.82                   -50.0
9           10          5551.05             3361.34                    70.0
10          11          4853.43             5551.05                   -10.0
11          12          3850.91             4853.43                   -20.0


## Method 4: Pandas using function - calculate revenue growth percentage %

In [51]:
def get_monthly_revenue(orders):
    orders['order_month'] = orders['order_date'].dt.to_period('M').dt.to_timestamp()
    monthly_revenue = (
        orders.groupby('order_month')
        .agg(monthly_revenue=('order_amount', 'sum'))
        .reset_index()
    )
    return monthly_revenue

monthly_revenue = get_monthly_revenue(orders)
print(monthly_revenue)

def calculate_revenue_growth(monthly_revenue):
    monthly_revenue['previous_monthly_revenue'] = monthly_revenue['monthly_revenue'].shift(1)
    monthly_revenue['revenue_growth_revenue'] = ((monthly_revenue['monthly_revenue'].fillna(0) - monthly_revenue['previous_monthly_revenue']).fillna(0) / 
                                                monthly_revenue['previous_monthly_revenue']).round(2) * 100
    return monthly_revenue

revenue_growth_percent = calculate_revenue_growth(monthly_revenue)
revenue_growth_percent
    

   order_month  monthly_revenue
0   2023-01-01          2051.37
1   2023-02-01          3225.72
2   2023-03-01          2441.02
3   2023-04-01          5266.72
4   2023-05-01          2474.76
5   2023-06-01          5196.00
6   2023-07-01          5013.02
7   2023-08-01          6381.82
8   2023-09-01          3361.34
9   2023-10-01          5551.05
10  2023-11-01          4853.43
11  2023-12-01          3850.91


,order_month,monthly_revenue,previous_monthly_revenue,revenue_growth_revenue
0,2023-01-01,2051.37,NaN,NaN
1,2023-02-01,3225.72,2051.37,57.0
2,2023-03-01,2441.02,3225.72,-24.0
3,2023-04-01,5266.72,2441.02,116.0
4,2023-05-01,2474.76,5266.72,-53.0
5,2023-06-01,5196.00,2474.76,110.0
6,2023-07-01,5013.02,5196.00,-4.0
7,2023-08-01,6381.82,5013.02,27.0
8,2023-09-01,3361.34,6381.82,-47.0
9,2023-10-01,5551.05,3361.34,65.0
